In [1]:
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta
import classes_func
from classes_func import *

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Creating a search instance with Search('City',start_day,end_day)
Barcelona = Search('Barcelona','2024-02-09','2024-02-13')

In [3]:
#Remove Cookies
#Find destination button
Barcelona.browser.find_element(by='xpath',value= '//button[@id="onetrust-reject-all-handler"]').click()

In [4]:
#Remove Google Sign in
# Define your wait timeout (adjust as needed)
wait_timeout = 10
wait = WebDriverWait(Barcelona.browser, wait_timeout)

try:
    # Switch to the iframe by index (if needed, adjust the index)
    Barcelona.browser.switch_to.frame(0)  # You may need to adjust the index based on your HTML structure

    # Now you can interact with elements within the iframe
    # Wait for the "Not Now" button to be clickable and click it
    not_now_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='TvD9Pc-Bz112c ZYIfFd-aGxpHf-FnSee' and @aria-label='Cerrar']")))
    not_now_button.click()

except Exception as e:
    print(f"An error occurred while handling the Google Sign-In pop-up: {e}")

finally:
    # Switch back to the default content
    Barcelona.browser.switch_to.default_content()


In [5]:
#Inputing city
Barcelona.input_city()

In [6]:
Barcelona.open_date_box()

In [7]:
#Select Date_start
Barcelona.date_selector(Barcelona.start_day)

In [8]:
#Select Date_end
Barcelona.date_selector(Barcelona.end_day)

In [9]:
#Search for Results
Barcelona.search_results()

In [10]:
time.sleep(5)
# Removing the Genius pop-up only requires a clicking outside the pop-up
obscured = check_obscures(Barcelona.browser,'//div[@class="efdb2b543b e4b7a69a57"]',"xpath")
try:
    if not obscured:
        Barcelona.browser.find_element(by='xpath',value= '//div[@class = "abcc616ec7 cc1b961f14 c180176d40 f11eccb5e8 ff74db973c"]//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e f4552b6561"]').click()
    else:
        print('no element blocking the path')
except Exception as e:
    print(f"An error occurred: {e}")
#html.b_firefox.b_firefox_121.supports_inline-block.supports_flexbox_unprefixed.supports_fontface.supports_hyphens.hasJS body#b2searchresultsPage.bookings2.b2.searchresults.es.lang_is_ltr.header_reshuffle.no_bg_img.nobg.user_center.app_user_center.sb_gradient_border.b-sprite-3.ds-traveller-header.lx_cwv_font_swap.bigblue_std_sm.bigblue_std_lg.system-font.iq-x-bar.iq-x-bar-new div.b9720ed41e.cdf0a9297c div.eb33ef7c47 div.f0c216ee26.c676dd76fe.b5018b639f div.c0528ecc22 div.dd5dccd82f div.ffd93a9ecb.dc19f70f85.eb67815534 div.abcc616ec7.cc1b961f14.c180176d40.f11eccb5e8.ff74db973c button.a83ed08757.c21c56c305.f38b6daa18.d691166b09.ab98298258.deab83296e.f4552b6561

Message: Element <div class="efdb2b543b e4b7a69a57"> is not clickable at point (779,247) because another element <div class="f7c2c6294c"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:333:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:176:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31



In [11]:
#Total number of Result pages
pages = Barcelona.result_pages()
pages

30

In [12]:
#Barcelona.scrape_results(pages)
# Also if it's going to slow, adjust the waiting times in the class_function
Barcelona.scrape_results(pages)#Add the number of pages you'll like to scrape

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29


In [13]:
Barcelona.df.shape

(586, 8)

In [14]:
import concurrent.futures
from tqdm import tqdm

descriptions = []
linkey = Barcelona.df['Link']

def scrape_description(link):
    URL = f'{link}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Getting Data
    div2 = soup.find('div', {'id': 'hp_hotel_name'})
    div = soup.find('div', {'id': 'property_description_content'})
    
    # Find all p elements with a specific class within the div
    if div is not None:
        specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
        if div2 is not None:
            # Find all h2 elements with a specific class within div2
            specific_class_h_elements = div2.find_all('h2', class_='d2fee87262 pp-header__title')
        else:
            specific_class_h_elements = []
    
        descriptions = []
        for h, p in zip(specific_class_h_elements, specific_class_p_elements):
            descriptions.append((h.get_text(strip=True), p.get_text(strip=True)))
    else:
        # If 'div' is not found, return an empty list
        descriptions = []
    
    return descriptions

threads = os.cpu_count()
with concurrent.futures.ThreadPoolExecutor(max_workers=threads-2) as executor:
    results = list(tqdm(executor.map(scrape_description, linkey), total=len(linkey), desc="Processing Links"))

# Flatten the results list
description_result = [item for sublist in results for item in sublist]
description_df = pd.DataFrame(description_result, columns=['Hotels', 'Descriptions'])


Processing Links: 100%|████████████████████████████████████████████████████████████████████████████| 586/586 [07:08<00:00,  1.37it/s]


In [15]:
description_df.shape

(586, 2)

In [16]:
# Apply left join between main df and desscriptions
Barcelona.df = pd.merge(Barcelona.df, description_df, how='left', on='Hotels')

In [17]:
Barcelona.df = Barcelona.df.drop_duplicates()
Barcelona.df[0:50]

,Hotels,Distance,District,Price,Rating,Stars,Comments,Link,Descriptions
0,Room Mate Gerard,"a 0,7 km del centro","Eixample, Barcelona",€ 852,"8,8",4 de 5,2.785 comentarios,https://www.booking.com/hotel/es/room-mate-ger...,El Room Mate Gerard en Barcelona ofrece alojam...
2,Sonder Casa Luz,a 450 m del centro,"Eixample, Barcelona",€ 690,"8,4",4 de 5,2.669 comentarios,https://www.booking.com/hotel/es/casa-luz-barc...,El Sonder Casa Luz se encuentra en Barcelona y...
4,Wilson Boutique Hotel,"a 1,8 km del centro","Sarrià-Sant Gervasi, Barcelona",€ 419,"8,5",4 de 5,2.127 comentarios,https://www.booking.com/hotel/es/wilson.es.htm...,El Wilson Boutique Hotel está situado en la av...
5,"Hotel Granados 83, a Member of Design Hotels","a 1,2 km del centro","Eixample, Barcelona",€ 592,"8,5",4 de 5,1.575 comentarios,https://www.booking.com/hotel/es/granados-83.e...,"El Granados 83, situado a 250 metros de la est..."
6,Hotel Derby,"a 2,4 km del centro","Les Corts, Barcelona",€ 396,"8,3",4 de 5,2.170 comentarios,https://www.booking.com/hotel/es/derby.es.html...,El Derby está situado a 300 metros del centro ...
7,NH Sants Barcelona,"a 2,7 km del centro","Les Corts, Barcelona",€ 513,"8,5",4 de 5,3.061 comentarios,https://www.booking.com/hotel/es/nh-sants-barc...,El NH Sants Barcelona está situado a 400 metro...
8,"Hotel Balmes, a member of Preferred Hotels & R...",a 1 km del centro,"Eixample, Barcelona",€ 549,"8,4",4 de 5,3.414 comentarios,https://www.booking.com/hotel/es/balmesderby.e...,El Balmes se encuentra en el centro de Barcelo...
9,Pol & Grace Hotel,"a 2,5 km del centro","Sarrià-Sant Gervasi, Barcelona",€ 426,"8,4",4 de 5,2.519 comentarios,https://www.booking.com/hotel/es/guillermo-tel...,El Pol & Grace Hotel se encuentra a solo 100 m...
10,Hotel Paxton Barcelona,"a 2,9 km del centro","Sant Martí, Barcelona",€ 615,"8,7",4 de 5,5.235 comentarios,https://www.booking.com/hotel/es/paxton-barcel...,Hotel Paxton Barcelona se encuentra en Barcelo...
11,Acta CITY47,"a 2,4 km del centro","Sants-Montjuïc, Barcelona",€ 517,"8,5",4 de 5,1.331 comentarios,https://www.booking.com/hotel/es/citypark.es.h...,"El Acta CITY47 está situado en Barcelona, a 5 ..."


In [18]:
#define downloads path
downloads_path = "./downloads_datasets/"
barcelona_09_13_data = Barcelona.df

barcelona_09_13_data.to_csv(os.path.join(downloads_path, "barcelona_09_13_data.csv"), index=False)

In [19]:
Barcelona.browser.quit()